###ADLS Gen2 storage capacity audit
This notebook calculates the storage size at folder level and persists results to a delta table

In [0]:
from pyspark.sql.functions import *
from multiprocessing.pool import ThreadPool
from multiprocessing import Value, Lock

Use locks to get the counts correctly in a multithread enviornment

In [0]:
counter_lock = Lock()
def fileIncrement():
    with counter_lock:
        fileCounter.value += 1
    return fileCounter.value

In [0]:
def getTableSizeIterator(path, FileSizeList):
  tableFileList = dbutils.fs.ls(path)
  if len(tableFileList) > 0:
    cnt = fileIncrement()
    print("paths: ", path, "cnt:", cnt, "cntFiles: ", len(tableFileList))
    tableFileDF = spark.createDataFrame(data = tableFileList, schema = ["path", "name","size"])
    FileSizeDF = tableFileDF.agg(count('*').alias('fileCount'),round((sum('size')/(1073741824)),3).alias('sizeGB'))
    FileSizeDF = FileSizeDF.withColumn('timestamp',current_timestamp()).withColumn('path',lit(path))
    FileSizeDF = FileSizeDF.select("path","timestamp","fileCount","sizeGB")
    FileSizeTempList = FileSizeDF.collect()
    FileSizeList += FileSizeTempList

    dirDF = tableFileDF.select("path").where("size = 0 and name not like '%.%' and name not like '%delta_log/' and name not like '%our=%' and name not like '%inute=%' and name not like '$_%' ESCAPE '$'")
    for path in dirDF.rdd.map(lambda line: "|".join([str(x) for x in line])).collect():
      getTableSizeIterator(path, FileSizeList)

  return FileSizeList

In [0]:
def getFolderSizePool(row):
  print ("getting size of: ", row.path)
  fileCounter = Value('i',0) 
  FinalList = []
  FinalList = getTableSizeIterator(row.path, FinalList)
  FileSizeCheckDF = spark.createDataFrame(data = FinalList, schema = ["path","timestamp","fileCount","sizeGB"]) 
  FileSizeCheckDF.write.mode("append").format("delta").saveAsTable("Lake_File_Details")

In [0]:
fileCounter = Value('i',0)
tableFileList = dbutils.fs.ls("dbfs:/mnt/lake/bronze")

#multithreading to get the stats parallelly.
pool=ThreadPool(10)
pool.map(getFolderSizePool,tableFileList)

In [0]:
%sql
select substring_index(path,'/',5) as path
  ,count(*),sum(fileCount) as fileCount
  ,round(sum(sizeGB),2) as sizeGB 
from  Lake_File_Details 
group by 1
order by 4 desc